In [1]:
import pandas as pd

In [7]:
df_bn5 = pd.read_csv("data/historical/BankNiftyFut-Sep-5min.csv", sep="\t", parse_dates=['Date Time'], encoding="utf_16_LE") 

#### Remove last empty column and set date time as index

In [8]:
df_bn5 = df_bn5[df_bn5.columns[:-1]]
df_bn5.set_index('Date Time', inplace=True)

#### Apply rolling mean columns for data

In [15]:
df_bn5['SMA6'] = df_bn5['Close'].rolling(6).mean()
df_bn5['SMA21'] = df_bn5['Close'].rolling(21).mean()

In [16]:
df_bn5.loc['2020-09-03']

,Open,Low,High,Close,Volume,SMA6,SMA21
Date Time,,,,,,,
2020-09-03 09:15:00,23938.80,23800.05,24079.80,23912.05,360576,23920.891667,23864.230952
2020-09-03 09:20:00,23924.00,23845.75,23924.00,23861.95,169050,23909.550000,23873.328571
2020-09-03 09:25:00,23860.60,23840.00,23895.40,23863.40,89575,23902.825000,23883.814286
2020-09-03 09:30:00,23863.40,23815.00,23870.10,23855.10,99700,23891.366667,23892.178571
2020-09-03 09:35:00,23858.00,23850.50,23906.70,23875.00,89075,23881.375000,23899.369048
...,...,...,...,...,...,...,...
2020-09-03 15:10:00,23599.00,23510.00,23605.00,23516.00,166925,23618.466667,23693.414286
2020-09-03 15:15:00,23514.00,23452.95,23520.25,23517.80,236325,23586.400000,23684.390476
2020-09-03 15:20:00,23517.00,23515.05,23580.00,23561.80,156375,23566.266667,23678.280952


In [11]:
df_bn5.head(5)

,Open,Low,High,Close,Volume
Date Time,,,,,
2020-08-21 09:15:00,22254.05,22131.80,22254.05,22184.05,16526
2020-08-21 09:20:00,22185.35,22185.35,22241.15,22240.30,16100
2020-08-21 09:25:00,22235.90,22221.85,22261.90,22255.55,12800
2020-08-21 09:30:00,22258.10,22256.40,22298.40,22275.90,12025
2020-08-21 09:35:00,22273.95,22256.45,22278.75,22269.00,9800


## Input the evaluation date below (1st friday of every month)

In [ ]:
eval_date = "2020-05-08"

### Fetch last year data

#### [Below data fetch will throw error if there is no trading day between start and end dates]

In [ ]:
start_date=eval_date
end_date = (pd.to_datetime(start_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
data = yf.download(momentum_stock_list, start = start_date, end=end_date)

In [ ]:
this_year_close = pd.DataFrame(data.iloc[0]['Adj Close'])

In [ ]:
this_year_close

In [ ]:
start_date=(pd.to_datetime(eval_date) - pd.DateOffset(years=1)).strftime('%Y-%m-%d')
end_date = (pd.to_datetime(start_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')

In [ ]:
start_date

In [ ]:
end_date

### Fetch last year data

#### [Below data fetch will throw error if there is no trading day between start and end dates]

In [ ]:
data = yf.download(momentum_stock_list, start = start_date, end=end_date)

In [ ]:
last_year_close = pd.DataFrame(data.iloc[0]['Adj Close'])

In [ ]:
last_year_close

In [ ]:
final_df = last_year_close.join(this_year_close)

In [ ]:
final_df

In [ ]:
final_df.rename(columns =lambda t: t.strftime('%Y-%m-%d'), inplace=True)

## Calculate Momentum

In [ ]:
last_year_date = final_df.columns[0]
this_year_date = final_df.columns[1]
final_df['Change'] = final_df[this_year_date] - final_df[last_year_date]

In [ ]:
final_df['% Change'] = (final_df['Change'] / final_df[last_year_date])*100

## Final Output

In [ ]:
final_df.sort_values(by=['% Change'], ascending=False, inplace=True)

In [ ]:
final_df.rename_axis('MOMENTUM_ATH', inplace=True)
final_df.rename_axis('', axis='columns', inplace=True)

In [ ]:
final_df.round(2).head(50)

In [ ]:
filename = f'data/output/momentum_ath_output_{eval_date}.xlsx'

In [ ]:
final_df.round(2).to_excel(filename)

In [ ]:
f"Done. Output file path and name --> {filename}"